In [1]:
from __future__ import annotations

from xdsl.dialects.builtin import ModuleOp

from toy_to_riscv.helpers import parse_toy, print_module

from riscv.emulator_iop import run_riscv, print_riscv_ssa

import toy.dialect as td
import riscv.riscv_ssa as rd
import toy_to_riscv.dialect as trd

### WIP

example = """
def main() {
  var a<2, 3> = [[1, 2, 3], [4, 5, 6]];
  var b<6> = [1, 2, 3, 4, 5, 6];
  var c<2, 3> = b;
  var d = a + c;
  print(d);
}
"""

### WIP

module = parse_toy(example)
print_module(module)
print()

### WIP

module = ModuleOp.from_region_or_ops([
    rd.DataSectionOp.from_ops([
        rd.LabelOp.get("main.a"),
        rd.DirectiveOp.get(".word", "0xA, 0x2, 0x2, 0x3, 0x6, 0x1, 0x2, 0x3, 0x4, 0x5, 0x6"),
        rd.LabelOp.get("main.b"),
        rd.DirectiveOp.get(".word", "0x9, 0x1, 0x6, 0x6, 0x1, 0x2, 0x3, 0x4, 0x5, 0x6"),
        rd.LabelOp.get("main.0"),
        rd.DirectiveOp.get(".word", "0x2, 0x2, 0x3"),
    ]),
    rd.FuncOp.from_ops('main', [
        heap := rd.LIOp.get('heap'),
        a := rd.LIOp.get('main.a'),
        b := rd.LIOp.get('main.b'),
        c_shape := rd.LIOp.get('main.0'),
        c := trd.ReshapeTensorOp.get(b, c_shape, heap),
        d := trd.AddTensorOp.get(a, c, heap),
        trd.PrintTensorOp.get(d),
        rd.ReturnOp.get(),
    ])
])

print_module(module)
print()

### WIP

code = print_riscv_ssa(module)
print(code)
print()

### WIP

from toy_to_riscv.accelerator import ToyAccelerator

run_riscv(code, extensions=[ToyAccelerator], unlimited_regs=True)


"builtin.module"() ({
  "toy.func"() ({
    %0 = "toy.constant"() {"value" = dense<[[1, 2, 3], [4, 5, 6]]> : tensor<2x3xi32>} : () -> tensor<2x3xi32>
    %1 = "toy.reshape"(%0) : (tensor<2x3xi32>) -> tensor<2x3xi32>
    %2 = "toy.constant"() {"value" = dense<[1, 2, 3, 4, 5, 6]> : tensor<6xi32>} : () -> tensor<6xi32>
    %3 = "toy.reshape"(%2) : (tensor<6xi32>) -> tensor<6xi32>
    %4 = "toy.reshape"(%3) : (tensor<6xi32>) -> tensor<2x3xi32>
    %5 = "toy.add"(%1, %4) : (tensor<2x3xi32>, tensor<2x3xi32>) -> tensor<2x3xi32>
    "toy.print"(%5) : (tensor<2x3xi32>) -> ()
    "toy.return"() : () -> ()
  }) {"sym_name" = "main", "function_type" = () -> ()} : () -> ()
}) : () -> ()

"builtin.module"() ({
  "riscv.data_section"() ({
    "riscv_ssa.label"() {"label" = #riscv.label<main.a>} : () -> ()
    "riscv_ssa.directive"() {"directive" = ".word", "value" = "0xA, 0x2, 0x2, 0x3, 0x6, 0x1, 0x2, 0x3, 0x4, 0x5, 0x6"} : () -> ()
    "riscv_ssa.label"() {"label" = #riscv.label<main.b>} : () -> ()
